# Lazy RDDs

### Introduction

So we saw in the last lesson that logs each of the steps taken when we transform or query our data.  It turns out that Spark plans out what steps are needed to be taken in advance, and then executes the function.  This way, with some planning, it can efficiently operate on our data. 

We'll learn more about how spark achieves this efficiency in this lesson.

### Getting Setup (On Google Colab)

* Begin by installing some pip packages and the java development kit.

In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

* Then set the java environmental variable

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

* Then connect to a SparkSession, setting the spark ui port to `4050`.

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("films").setMaster("local[2]")
sc = SparkContext.getOrCreate(conf=conf)

* Then we need to install ngrok which will allow us to place our local spark ui on the web.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')

* And finally we get a link our Spark UI

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Looking Under the Hood

Now  let's again create an RDD from our movie records.

In [ ]:
movies = ['dark knight', 'dunkirk', 'pulp fiction', 'avatar']

In [ ]:
movies_rdd = sc.parallelize(movies)
movies_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

And then let's capitalize the movies, and select the movies that begin with `d`.

In [ ]:
movies_rdd.map(lambda movie: movie.title()).collect()

['Dark Knight', 'Dunkirk', 'Pulp Fiction', 'Avatar']

Now as we know, Spark will partition the dataset across the cores of the executors, and then map through the records in parallel, returning all of the results.

> <img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/parallel.png?raw=1" width="60%">

Now let's change the function so that this time, instead of returning all of the results, we just return the first result.

In [ ]:
movies_rdd.map(lambda movie: movie.title()).take(1)

['Dark Knight']

Now if we think about, this previous step, here we would not have to map through all of the steps just to return a single result.  And it turns out if we look at Spark, we can see that even though the dataset was distributed -- it only needed to perform work on a single partition to return one result.

> <img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/individual_task.png?raw=1" width="80%">

This ability, to see the end result that needs to be returned, and to work efficiently to only take the needed steps to return those results, is a valuable feature when working with large datasets.  And we can better see how Spark accomplishes it in the next section.

### A little experiment

If we run the code below, notice that nothing is returned.

In [ ]:
movies_rdd.map(lambda movie: movie.title())

PythonRDD[8] at RDD at PythonRDD.scala:53

And even if we chain the map and the filter methods, still nothing is returned.

In [ ]:
movies_rdd.map(lambda movie: movie.title()).filter(lambda movie: movie[0] == 'd')

PythonRDD[3] at RDD at PythonRDD.scala:53

It's only when we add a collect function on the end, will some data be returned.

In [ ]:
movies_rdd.filter(lambda movie: movie[0] == 'd').map(lambda movie: movie.title()).collect()

['Dark Knight', 'Dunkirk']

So above, nothing was returned when we ran the `map` and `collect` functions, because when we only executed those functions, Spark did not actually act on the data.  Then in the third line we finally did act on the data.  We told Spark that we want to both transform, and filter the data, and then return all of the results.  

So it's only when we called the `collect` function that Spark's driver determined the tasks to then send off to the executors and return the results.

### Transformations and Actions

So above we can see that the functions `map` and `filter` do not actually perform any work on our data.  Instead steps are only kicked off when we call the `collect` method.  

In Spark, the methods that kick off tasks and return results are called **actions** (eg. collect).  And methods like `map` and `transform` that do not are called **transformations**.  

1. Transformations

So we already saw that transformations include `map` and `filter`, and our transformations do not actually return results to our users.  Here's a couple other transformations.

* sample

The `sample` method allows us to take a random sample from our dataset.  

In [ ]:
movies_rdd.sample(fraction = .2, withReplacement = True)

PythonRDD[16] at RDD at PythonRDD.scala:53

> Notice that it does not return any data.

* distinct

In [ ]:
movies_rdd.distinct()

PythonRDD[14] at RDD at PythonRDD.scala:53

> Distinct finds the unique results.  Notice that it also does not return data.

Finally, we have already seen `map`, which provides a one to one transformation of our records, and `select` which filters our data.  In each case, our transformations do not return data to us. 

2. Actions

Actions are a bit more about the end result.  So far we've learned about `collect`, which returns *all* of the results of a series of transformations.  

* Take

We've also seen `take`, which limits our results to a subset.

In [ ]:
movies_rdd.distinct().take(2)

['dark knight', 'dunkirk']

> So `take` is similar to the `LIMIT` function in SQL. Notice that here our records are returned.

* Count

In [ ]:
movies_rdd.distinct().count()

4

Count simply counts the results.

### Summary

So we can see that, our actions have a bit of finality to them.  To get a better sense of the transformation and action functions, it's worth looking at the [documentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations).

### Resources

[Berkley White Paper](https://www.usenix.org/system/files/conference/nsdi12/nsdi12-final138.pdf)

[Pyspark RDD Methods blog](https://www.nbshare.io/notebook/403283317/How-To-Analyze-Data-Using-Pyspark-RDD/)


[Databricks Debugging Spark Streaming](https://docs.databricks.com/spark/latest/rdd-streaming/debugging-streaming-applications.html)